# Calibracao do modulo de contagem

In [1]:
import os
import cv2
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from ultralytics import YOLO

In [2]:
def createNameImage(paths_image):
    name_format = paths_image.split("\\")[-1]
    image_name=name_format.split(".")[0] 
    
    return image_name

#Cria uma Lista com o Nome de todas as imagens no path de entrada 
def createListNameImageFiles(input_path):

    from pathlib import Path
    images = Path(input_path).glob("*.jpg")
    image_names_list = [str(p) for p in images]

    return image_names_list



# Esta função recebe um path da imagem, carrega e retorna uma imagem BGR 
def loadImage(IMAGE_PATH):
    print("Carregando imagem " + IMAGE_PATH)
    imagem = cv2.imread(IMAGE_PATH)
    
    return imagem

# Esta função recebe uma imagem e realiza a vizualização
def showImage(imagem):
    print("Visualizando a imagem...")
    plt.imshow(cv2.cvtColor(imagem, cv2.COLOR_BGR2RGB))
    plt.show()
    
# Esta função recebe o caminho do modelo e carrega e retorna um modelo yolo de detecção de objetos
def loadYOLOModel(MODEL_PATH):
    model = YOLO(MODEL_PATH)
    print("Carregando modelo de detecção YOLO " + MODEL_PATH)
    return model

# Esta função recebe uma imagem e as dimenções de largura e altura das janelas. Por fim,retorna um vetor de janelas dessa imagem
def generateWindows(imagem_orig, WIN_WIDTH, WIN_HEIGHT):
    altura, largura, canal = imagem_orig.shape
    
    n_largura = int(largura/WIN_WIDTH)
    n_altura = int(altura/WIN_HEIGHT)
    
    
    i = 1
    x1 = 0
    y1 = 0
    x2 = WIN_WIDTH
    y2 = WIN_HEIGHT
    
    windows_vetor = []
    
    for a in range(n_altura):
        for l in range(n_largura):
            
            #img_draft = cv2.rectangle(imagem_orig, (x1, y1), (x2, y2), (255,0,0), 4)
            #showImage(img_draft)
            windows_img = imagem_orig[y1:y2, x1:x2]
            windows_vetor.append(windows_img)
            #showImage(windows_img)
            
            
            i = i + 1
            x1 = x1 + WIN_WIDTH
            x2 = x2 + WIN_WIDTH
            l = l + 1           
        
        x1 = 0
        y1 = y2
        x2 = WIN_WIDTH
        y2 = y2 + WIN_HEIGHT
        a = a + 1

    return windows_vetor

# Esta função recebe uma imagem em janela, ID da janela, o modelo de detecção e o valor threshold para realizar a detecção e
# contagem dos objetos existentes na janela. Por fim, retorna o numero de vacas detectadas na janela e também retorna um 
# um Dataframe com a posição das detecções.

def detectAndCountWindows(win_image, win_ID, model, DETECTION_THRESHOLD):
    frame = win_image
    results = model(win_image, verbose=False)[0]
    
    class_name_dict = {0: 'cow', 1:'noCow'}
    
    cow_number = 0
    
    row_list = []
    
    for result in results.boxes.data.tolist():
        x1, y1, x2, y2, score, class_id = result
            
        if int(class_id) != 0:
            pass

        else:

            if score > DETECTION_THRESHOLD:
                cow_number = cow_number + 1
                
                #cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
                #cv2.putText(frame, class_name_dict[int(class_id)].upper()+"_"+str(round(score)*100)+"%", (int(x1), int(y1 - 10)),
                 #           cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 255, 0), 3, cv2.LINE_AA)

                value = [win_ID, 'cow', x1, y1, x2, y2, score]
                row_list.append(value)

                #plt.imshow(frame)
                #plt.show()
                

    column_name = ['win_ID', 'classe', 'x1', 'y1', 'x2', 'y2', 'scores']
    detections_df = pd.DataFrame(row_list, columns=column_name)
                
    return cow_number, detections_df

# Esta função recebe o vetor de janelas, o modelo e o threshold. Em seguida, Retorna a contagem total de vacas e um Dataframe 
# com as posições
def countingCows(windows_vetor, model, DETECTION_THRESHOLD):
    
    win_ID = 0
    vetor_qt_cows = []
    
    row_list = []
    column_name = ['win_ID', 'classe', 'x1', 'y1', 'x2', 'y2', 'scores']
    final_detection_df = pd.DataFrame(row_list, columns=column_name)
    
    for win_image in windows_vetor:
        
        cow_number, detections_df = detectAndCountWindows(win_image, win_ID, model, DETECTION_THRESHOLD)
        
        vetor_qt_cows.append(cow_number)
        final_detection_df = pd.concat([final_detection_df, detections_df])
        
        win_ID = win_ID + 1
        
        
    qt_cows = sum(vetor_qt_cows)
    
    return qt_cows, final_detection_df.reset_index(drop=True)

def convetBBoxWinToBBoxOrig(win_x1, win_y1, win_x2, win_y2, win_ID, WIN_WIDTH, WIN_HEIGHT, n_largura, n_altura):
    
    i = 0
    x1 = 0
    y1 = 0
    x2 = WIN_WIDTH
    y2 = WIN_HEIGHT
    
    windows_vetor = []
    
    for a in range(n_altura):
        for l in range(n_largura):
            
            if i == win_ID:
                orig_x1 = x1 + win_x1
                orig_y1 = y1 + win_y1
                orig_x2 = x1 + win_x2
                orig_y2 = y1 + win_y2
                
                return orig_x1, orig_y1, orig_x2, orig_y2

            
            
            i = i + 1
            x1 = x1 + WIN_WIDTH
            x2 = x2 + WIN_WIDTH
            l = l + 1           
        
        x1 = 0
        y1 = y2
        x2 = WIN_WIDTH
        y2 = y2 + WIN_HEIGHT
        a = a + 1
    
    
    return orig_x1, orig_y1, orig_x2, orig_y2

def writeNumCow(image, qt_total_cow):
    # font
    font = cv2.FONT_HERSHEY_SIMPLEX

    # org
    org = (100, 200)

    # fontScale
    fontScale = 5

    # Blue color in BGR
    color = (0, 255, 0)

    # Line thickness of 2 px
    thickness = 10

    # Using cv2.putText() method
    image = cv2.putText(image, "Numero de Vacas: " + str(qt_total_cow), org, font, 
                       fontScale, color, thickness, cv2.LINE_AA)
    
    return image
    
    
def printDetections(imagem_orig, qt_cows, final_detection_df, WIN_WIDTH, WIN_HEIGHT):
    
    altura, largura, canal = imagem_orig.shape
    
    n_largura = int(largura/WIN_WIDTH)
    n_altura = int(altura/WIN_HEIGHT)
    
    
    n_detections = len(final_detection_df)
    for i in range(n_detections):
        
        win_ID = final_detection_df.iloc[i][0]
        classe = final_detection_df.iloc[i][1]
        win_x1 = final_detection_df.iloc[i][2]
        win_y1 = final_detection_df.iloc[i][3]
        win_x2 = final_detection_df.iloc[i][4]
        win_y2 = final_detection_df.iloc[i][5]
        score = final_detection_df.iloc[i][6]
        
        x1, y1, x2, y2 = convetBBoxWinToBBoxOrig(win_x1, win_y1, win_x2, win_y2, win_ID, WIN_WIDTH, WIN_HEIGHT, n_largura, n_altura)

        cv2.rectangle(imagem_orig, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 10)
        cv2.putText(imagem_orig, classe + "_" + str(i+1), (int(x1), int(y1 - 10)), cv2.FONT_HERSHEY_SIMPLEX, 2.5, (0, 255, 0), 8, cv2.LINE_AA)
        
    
    writeNumCow(imagem_orig, qt_cows)
    showImage(imagem_orig)
    

In [3]:
#Threshold
THRESHOLD_LIST=np.arange(0.60, 1, 0.01).tolist()

WIN_WIDTH = 500
WIN_HEIGHT = 600

MODEL_PATH = os.path.join('.', 'runs', 'detect', 'train4', 'weights', 'best.pt')

VALIDATION_IMAGES_PATH = "C:\\NEW_TCC\\dataset\\calibracao"

VALIDATION_ANOTATION_DF_PATH = os.path.join(VALIDATION_IMAGES_PATH, "otimization_annotation_df.csv")

CALIBRATION_DF_PATH = os.path.join(VALIDATION_IMAGES_PATH, "calibration_df.csv")

In [4]:
val_image_list = createListNameImageFiles(VALIDATION_IMAGES_PATH)
val_anotation_df = pd.read_csv(VALIDATION_ANOTATION_DF_PATH)

model = loadYOLOModel(MODEL_PATH)

Carregando modelo de detecção YOLO .\runs\detect\train4\weights\best.pt


In [5]:
i=1
for image_path in val_image_list:
    for threshold in THRESHOLD_LIST:
        i=i+1
        
print(i) 

1001


In [6]:
win_width = WIN_WIDTH
win_height = WIN_HEIGHT

row_list=[]
n=1
for image_path in val_image_list:
    name_image = createNameImage(image_path)
    df=val_anotation_df[val_anotation_df['filename']==name_image]
    qt_val = len(df)
    imagem_orig = loadImage(image_path)

    for threshold in THRESHOLD_LIST: 

        windows_vetor = generateWindows(imagem_orig, win_width, win_height)

        qt_cows, final_detection_df = countingCows(windows_vetor, model, threshold)

        qt_pred = qt_cows

        row_list.append([name_image, win_width, win_height, threshold, qt_val, qt_pred])
        n=n+1
        print(n)
                

                

column_name = ['filename', 'win_width', 'win_height', 'threshold', 'qt_val', 'qt_pred']
calibration_df = pd.DataFrame(row_list, columns=column_name) 

calibration_df.to_csv(CALIBRATION_DF_PATH,index=False)

Carregando imagem C:\NEW_TCC\dataset\calibracao\cow102.JPG
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
Carregando imagem C:\NEW_TCC\dataset\calibracao\cow11.JPG
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
Carregando imagem C:\NEW_TCC\dataset\calibracao\cow110.JPG
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
Carregando imagem C:\NEW_TCC\dataset\calibracao\cow111.JPG
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
Carregando imagem C:\NEW_TCC\dataset\calibracao\cow114.JPG
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
Carregando i

In [7]:
calibration_df

,filename,win_width,win_height,threshold,qt_val,qt_pred
0,cow102,500,600,0.60,7,7
1,cow102,500,600,0.61,7,7
2,cow102,500,600,0.62,7,7
3,cow102,500,600,0.63,7,7
4,cow102,500,600,0.64,7,7
...,...,...,...,...,...,...
995,cow63,500,600,0.95,7,0
996,cow63,500,600,0.96,7,0
997,cow63,500,600,0.97,7,0
998,cow63,500,600,0.98,7,0


In [8]:
df = calibration_df[calibration_df['qt_val']==calibration_df['qt_pred']]
df_2 = df.groupby(["win_width", "win_height", "threshold"]).size().reset_index(name="time")
df_3 = df_2.sort_values(by=['time', 'threshold', 'win_width', 'win_height'], ascending=False)
final_win_width = df_3.iloc[0][0]
final_win_height = df_3.iloc[0][1]
final_threshold = df_3.iloc[0][2]
final_time = df_3.iloc[0][3]


print("Os 5 melhores valores de hiperparametros")
df_3.head(5)

Os 5 melhores valores de hiperparametros


,win_width,win_height,threshold,time
11,500,600,0.71,14
10,500,600,0.70,14
9,500,600,0.69,14
12,500,600,0.72,13
8,500,600,0.68,13
